# <ins>Analysis on the Potential of Life on Exoplanets</ins>

## By: Ethan Barr and Tim Freerksen

## <ins>Introduction</ins>

Is there life in space? This has been a question for many years with no real evidence to back up the claim that there is. Our goal in this project is to take the exoplanets already discovered and see what the probability of life on these planets real is and hope that a good amount of the exoplanets that we discovered so far fit well into the criteria of haveing the ability to hold life.

Throughout this tutorial, we will try to see how many of the exoplanets we have discovered so far the right conditions in order to harbor life and then we will see what the probability is that a future exoplanet will end up holding life.

### <ins>Required Tools</ins>

The following libraries used for the project:

1. pandas
2. 

If having issues with python 3+ or panda, we recommend referring to these following websites for more information:

1. https://docs.python.org/3/
2. https://pandas.pydata.org/pandas-docs/stable/install.html

In [11]:
# All Imports
import pandas as pd

In [12]:
data = pd.read_csv('PS.csv')
data.set_index('loc_rowid', inplace=True)
print(data.head())

            pl_name hostname  default_flag  sy_snum  sy_pnum  discoverymethod  \
loc_rowid                                                                       
1          11 Com b   11 Com             0        2        1  Radial Velocity   
2          11 Com b   11 Com             1        2        1  Radial Velocity   
3          11 UMi b   11 UMi             0        1        1  Radial Velocity   
4          11 UMi b   11 UMi             0        1        1  Radial Velocity   
5          11 UMi b   11 UMi             1        1        1  Radial Velocity   

           disc_year                           disc_facility  \
loc_rowid                                                      
1               2007                        Xinglong Station   
2               2007                        Xinglong Station   
3               2009  Thueringer Landessternwarte Tautenburg   
4               2009  Thueringer Landessternwarte Tautenburg   
5               2009  Thueringer Landessternwart